In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import sys
import pathlib

PROJECT_ROOT = pathlib.Path().cwd().parent
sys.path.append(str(PROJECT_ROOT))

image_path = PROJECT_ROOT / 'ReMIND2Reg_dataset' / 'imagesTr'

from src.utils import *
from src.datatype import MRIDataset
from src.renderer import UltrasoundRenderer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib
import torch
import torch.nn.functional as F

In [7]:
patient_id = "003" # between 000 and 102, t1 and t2 don't exist for all patients
iUS_file = f"ReMIND2Reg_0{patient_id}_0000.nii.gz"
t1_file = f"ReMIND2Reg_0{patient_id}_0001.nii.gz"
t2_file = f"ReMIND2Reg_0{patient_id}_0002.nii.gz"

# Load the NIfTI file
img_iUS = nib.load(image_path / iUS_file)
img_t1 = nib.load(image_path / t1_file)
img_t2 = nib.load(image_path / t2_file)

# Extract image data as a numpy array
img_iUS_data = img_iUS.get_fdata()
img_t1_data = img_t1.get_fdata()
img_t2_data = img_t2.get_fdata()
print("Image shape:", img_iUS_data.shape)

slice_idx = img_iUS_data.shape[2] // 2

# plt.imshow(img_iUS_data[:, :, slice_idx], cmap="gray")
# plt.title(f"Slice {slice_idx} of the iUS volume")
# plt.axis("off")
# plt.show()

# plt.imshow(img_t1_data[:, :, slice_idx], cmap="gray")
# plt.title(f"Slice {slice_idx} of the MRI T1 volume")
# plt.axis("off")
# plt.show()

# plt.imshow(img_t2_data[:, :, slice_idx], cmap="gray")
# plt.title(f"Slice {slice_idx} of the MRI T2 volume")
# plt.axis("off")
# plt.show()

Image shape: (256, 256, 256)


In [11]:
patient_id = "003" # between 000 and 102, t1 and t2 don't exist for all patients
iUS_file = f"ReMIND2Reg_0{patient_id}_0000.nii.gz"
t1_file = f"ReMIND2Reg_0{patient_id}_0001.nii.gz"
t2_file = f"ReMIND2Reg_0{patient_id}_0002.nii.gz"
dataset = MRIDataset([image_path / t1_file], "T1", axis=0)

In [ ]:
sample = dataset[0]
volume = sample["image"].squeeze(0)          # (D,H,W)

# set up a fan of rays in the x-z plane (for a 2D slice)
N_rays = 128
theta_max = 30 * torch.pi / 180             # 30 degrees
angles = torch.linspace(-theta_max, theta_max, N_rays)
# rays lie in (x,z) plane: [cos, 0, sin]
directions = torch.stack([torch.sin(angles),
                          torch.zeros_like(angles),
                          torch.cos(angles)], dim=1)

# choose a source point
D, H, W = volume.shape
source = torch.tensor([80, 40, 0], dtype=torch.float32)

# simulate & plot
renderer = UltrasoundRenderer(num_samples=256, attenuation_coeff=0.8)
frame = renderer.simulate_frame(volume, source, directions)
# renderer.plot_frame(frame)
bmode = renderer.frame_to_bmode(frame)
renderer.plot_sector(frame, angles, spacing=1.0)
renderer.plot_bmode(bmode)
renderer.plot_sector_bmode(bmode, angles, spacing=1.0)



TypeError: arange() received an invalid combination of arguments - got (int, int, dtype=numpy.dtypes.Float32DType), but expected one of:
 * (Number end, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (Number start, Number end, *, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (Number start, Number end, Number step = 1, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
